In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [2]:
ls ${FD_RES}/region

annotation_cCREs           annotation_gencode  annotation_remap2022
annotation_enhancer_junke  annotation_motif    KS91_K562_ASTARRseq_peak_macs


In [3]:
ls ${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs

KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.ccre_all.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.gencode_v24_tss.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.nuc.bed.gz
KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.remap2022_nr.bed.gz
KS91_K562_hg38_ASTARRseq_Input.ccre_all.freq.tsv
KS91_K562_hg38_ASTARRseq_Input.remap2022_nr.tsv


In [4]:
ls ${FD_RES}/region/annotation_enhancer_junke

enhancer_junke_peak_astarr.bed.gz  enhancer_junke_peak_wstarr.bed.gz


In [7]:
FN_ANNS=($(ls ${FD_RES}/region/annotation_enhancer_junke))
for FN_ANN in ${FN_ANNS[@]}; do
    echo ${FN_ANN}
done

enhancer_junke_peak_astarr.bed.gz
enhancer_junke_peak_wstarr.bed.gz


## Annotation with Junke's enhancers

In [16]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_enhancer_junke_peak.%a.txt \
    --array 0-1 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/region/annotation_enhancer_junke
FN_ANNS=($(ls ${FD_RES}/region/annotation_enhancer_junke))
FN_ANN=${FN_ANNS[${SLURM_ARRAY_TASK_ID}]}
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/region/KS91_K562_ASTARRseq_peak_macs
FN_OUT=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.${FN_ANN}
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
zcat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29433942


In [17]:
ls ${FD_LOG}/annotation_astarr_peak_macs_input_enhancer_junke_peak.*.txt

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/annotation_astarr_peak_macs_input_enhancer_junke_peak.0.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/annotation_astarr_peak_macs_input_enhancer_junke_peak.1.txt


In [18]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_enhancer_junke_peak.0.txt

Hostname:           x1-01-4.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         12-17-22+16:09:18

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_enhancer_junke/enhancer_junke_peak_astarr.bed.gz

show first few lines of input
chr1	115570	115690
chr1	605280	605300
chr1	605330	605340
chr1	605350	605370
chr1	605440	605480


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.enhancer_junke_peak_astarr.bed.gz

show first few lines of output:
chr1	115411	115986	chr1	115570	115690	120
chr1	605090	605823	chr1	605280	605300	20
chr1	605090	605823	chr1	605330	605340	10
ch

In [19]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_enhancer_junke_peak.1.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  1
Time Stamp:         12-17-22+16:09:18

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/annotation_enhancer_junke/enhancer_junke_peak_wstarr.bed.gz

show first few lines of input
chr1	778410	778450
chr1	778460	778820
chr1	778830	778860
chr1	778870	779090
chr1	818900	818920


Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/region/KS91_K562_ASTARRseq_peak_macs/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.enhancer_junke_peak_wstarr.bed.gz

show first few lines of output:
chr1	777949	779437	chr1	778410	778450	40
chr1	777949	779437	chr1	778460	778820	360
chr1	777949	779437	chr1	778830	778860	30
ch